In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference')
print(os.getcwd())

/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference


In [2]:
import pprint
import pickle

import torch
from torch.nn import functional as F
import torch.optim as optim
from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np

from pose_pred import dataset
from pose_pred.pose_pred_network import PoseNetwork

In [3]:
testing_input_img_dir = "data/input_images/pytorch3d_shapenet_renders/03001627/rgba"
gt_pose_path = "../occupancy_networks/out/pytorch3d_renders/03001627/generation/meshes/rec_camera_params.pt"
load_model_weights_path = "notebooks/pose_net_weights_100_03001627.pt"

output_pred_pose_path = "data/predicted_poses/03001627_pred_camera_params.pt"

batch_size = 1
device = torch.device("cuda:0")
cfg = {"input_img_dir": testing_input_img_dir, "pose_path":gt_pose_path}

In [4]:
test_dataset = dataset.PosePredDataset(cfg)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=True, drop_last=False)

In [5]:
pose_net = PoseNetwork(cfg, device).to(device)
pose_net.load_state_dict(torch.load(load_model_weights_path))

<All keys matched successfully>

In [6]:
pred_poses_dict = {}
azim_differences = []
elev_differences = []

with torch.no_grad():
    for batch_i, batch in enumerate(tqdm(test_dataloader)):
        pose_net.eval()
        instance_name = batch["instance_name"][0]
        pred_pose = pose_net(batch)
        
        gt_pose = batch["pose"].to(device)
        pred_pose_offsets = torch.abs(gt_pose-pred_pose)
        azim_differences.append(pred_pose_offsets[:,0].detach().cpu())
        elev_differences.append(pred_pose_offsets[:,1].detach().cpu())
        
        pred_poses_dict[instance_name] = {"azim": pred_pose[0][0].item(), "elev":pred_pose[0][1].item(), "dist":gt_pose[0][2].item()}
        
azim_differences = torch.cat(azim_differences).numpy()
elev_differences = torch.cat(elev_differences).numpy()

In [7]:
pickle.dump(pred_poses_dict, open(output_pred_pose_path,"wb"))

In [8]:
print("mean azim offset: {} +/- {}".format(np.mean(azim_differences), np.std(azim_differences)))
print("mean elev offset: {} +/- {}".format(np.mean(elev_differences), np.std(elev_differences)))

mean azim offset: 8.6765775680542 +/- 30.4622802734375
mean elev offset: 0.5500335097312927 +/- 0.31314530968666077


In [9]:
pred_poses_dict = pickle.load(open(output_pred_pose_path, "rb"))
gt_pose_dict = pickle.load(open(gt_pose_path, "rb"))
print(len(pred_poses_dict))
print(len(gt_pose_dict))

for instance in gt_pose_dict:
    print("\n{}".format(instance))
    print("pr: {}".format(pred_poses_dict[instance]))
    print("gt: {}".format(gt_pose_dict[instance]))

1355
1355

f595abef9bc7320944b2fa2cac0778f5
pr: {'azim': 146.69253540039062, 'elev': 39.76948547363281, 'dist': 1.3333332538604736}
gt: {'azim': 152.59878540039062, 'elev': 40.0, 'dist': 1.3333332538604736}

d334e5ab55188d62680876614ed35a7f
pr: {'azim': 159.2704315185547, 'elev': 40.806396484375, 'dist': 1.2692307233810425}
gt: {'azim': 0.59326171875, 'elev': 40.0, 'dist': 1.2692307233810425}

cf32f38c2b6c58dce45ef4135c266a12
pr: {'azim': 328.7128601074219, 'elev': 39.18476486206055, 'dist': 1.4615384340286255}
gt: {'azim': 331.8652038574219, 'elev': 40.0, 'dist': 1.4615384340286255}

fae27953e0f0404ca99df2794ec76201
pr: {'azim': 311.6683654785156, 'elev': 39.28369140625, 'dist': 1.2692307233810425}
gt: {'azim': 317.1842041015625, 'elev': 40.0, 'dist': 1.2692307233810425}

f619416ed17ca24a5dc688388b7d266
pr: {'azim': 259.7403869628906, 'elev': 39.04014587402344, 'dist': 1.2692307233810425}
gt: {'azim': 357.9119873046875, 'elev': 40.0, 'dist': 1.2692307233810425}

d2304054a8dc7ea4febad4


f9bb302961c91d7220160736f02bc9e4
pr: {'azim': 129.4515838623047, 'elev': 39.47264099121094, 'dist': 1.3974359035491943}
gt: {'azim': 127.03506469726562, 'elev': 40.0, 'dist': 1.3974359035491943}

ee484fb2c1e4e611a0dcc52ca811e565
pr: {'azim': 29.681148529052734, 'elev': 39.7113037109375, 'dist': 1.2051281929016113}
gt: {'azim': 19.07016372680664, 'elev': 40.0, 'dist': 1.2051281929016113}

e7b1407e16d9d644c681404257d94ad9
pr: {'azim': 245.5498046875, 'elev': 39.029502868652344, 'dist': 1.3333332538604736}
gt: {'azim': 243.31631469726562, 'elev': 40.0, 'dist': 1.3333332538604736}

e102ff4df6f28b6cdb400c09abe26220
pr: {'azim': 50.09821319580078, 'elev': 39.82230758666992, 'dist': 1.5897436141967773}
gt: {'azim': 48.8748893737793, 'elev': 40.0, 'dist': 1.5897436141967773}

f66ea5e07a8ad9eb283b00891f680579
pr: {'azim': 216.6708221435547, 'elev': 39.59666442871094, 'dist': 1.4615384340286255}
gt: {'azim': 218.6511688232422, 'elev': 40.0, 'dist': 1.4615384340286255}

f6810de4042cc5ce57bd4bc6e

gt: {'azim': 282.3774108886719, 'elev': 40.0, 'dist': 1.2692307233810425}

e39df7339552595febad4f49b26ec52
pr: {'azim': 91.76730346679688, 'elev': 39.464073181152344, 'dist': 1.2692307233810425}
gt: {'azim': 90.04190826416016, 'elev': 40.0, 'dist': 1.2692307233810425}

f4e5698718f3d4494311a07b696e63e3
pr: {'azim': 56.75043487548828, 'elev': 39.935970306396484, 'dist': 1.5256410837173462}
gt: {'azim': 49.3812255859375, 'elev': 40.0, 'dist': 1.5256410837173462}

fa041b0276dcaaaf98e0d1738edd4f19
pr: {'azim': 70.15177917480469, 'elev': 39.591182708740234, 'dist': 1.2692307233810425}
gt: {'azim': 72.11629486083984, 'elev': 40.0, 'dist': 1.2692307233810425}

d8c6c9fd4919e7f1c8d01774acf97a
pr: {'azim': 353.4015197753906, 'elev': 39.67048645019531, 'dist': 1.3974359035491943}
gt: {'azim': 357.2012634277344, 'elev': 40.0, 'dist': 1.3974359035491943}

fc818d6fe03f098fd6f4cef762589739
pr: {'azim': 73.25821685791016, 'elev': 39.5247802734375, 'dist': 1.2051281929016113}
gt: {'azim': 78.37195587158